In [1]:
import pickle
import numpy as np
import pandas as pd
import warnings
import itertools
import random
import gc
import torch
from copy import deepcopy
from torch import nn
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_geometric.nn import GCNConv, Sequential
from tqdm.notebook import tqdm
from pathlib import Path
from dataclasses import dataclass, field
from matplotlib import pyplot as plt
import seaborn as sns
import wandb
from dataclasses import asdict

sns.set()

warnings.simplefilter("ignore")

## データセットを準備


In [2]:
rootdir = Path().resolve().parent.parent
inputdir = rootdir / "data" / "predict-ai-model-runtime"
node_feat_dir = rootdir / "data" / "google-slow-vs-fast-layout-7-85"
trans_node_feat_dir = rootdir / "data" / "google-slow-vs-fast-layout-6-79-dataset"
trans_node_config_feat_dir = rootdir / "data" / "google-slow-vs-fastlayout7-81-dataset"
workdir = Path().resolve() / "out"
workdir.mkdir(exist_ok=True, parents=True)

In [3]:
dataset_dict = {}
ignores = []
for ds in ["train", "valid", "test"]:
    records = []
    for arch, perm in itertools.product(["nlp", "xla"], ["default", "random"]):
        datadir = inputdir / f"npz_all/npz/layout/{arch}/{perm}/{ds}"
        for filepath in sorted(datadir.glob("*.npz")):
            filename = str(filepath).split("/")[-1].replace(".npz", "")

            if (ds != "test") and (("mlperf" in filename) or ("openai" in filename)):
                ignores.append(filepath)
                continue
            records.append(
                {
                    "arch": arch,
                    "perm": perm,
                    "filename": filename,
                    "filepath": filepath,
                    "node_feat_filepath": str(
                        node_feat_dir / arch / perm / ds / f"{filename}.npz"
                    ),
                    "trans_node_feat_filepath": str(
                        trans_node_feat_dir
                        / "layout"
                        / arch
                        / perm
                        / ds
                        / f"{filename}.npz"
                    ),
                    "trans_node_config_filepath": str(
                        trans_node_config_feat_dir
                        / arch
                        / perm
                        / ds
                        / f"{filename}.npz"
                    ),
                }
            )
    dataset_dict[ds] = pd.DataFrame(records)

In [4]:
# for filepath in tqdm(ignores):
#     node_config_feat = np.load(filepath)["node_config_feat"]

#     for i in range(1, node_config_feat.shape[0]):
#         if not (node_config_feat[0] == node_config_feat[i]).all():
#             filepath
#             break

In [5]:
dfcat = pd.DataFrame(
    [
        {"number": 0, "num_dims": 1, "num_cats": 19, "cats": list(range(19))},
        {"number": 1, "num_dims": 54, "num_cats": 6, "cats": list(range(6))},
    ]
)
dfcat.head()

,number,num_dims,num_cats,cats
0,0,1,19,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,1,54,6,"[0, 1, 2, 3, 4, 5]"


In [6]:
dfcat_config = pd.DataFrame(
    [
        {
            "number": 0,
            "num_dims": 18,
            "num_cats": 8,
        },  # output_layout, input_layout, kernel_layout
    ]
)
dfcat_config

,number,num_dims,num_cats
0,0,18,8


In [7]:
for ds in dataset_dict:
    for i, row in dataset_dict[ds].iterrows():
        np.load(row["filepath"])
        np.load(row["node_feat_filepath"])
        np.load(row["trans_node_feat_filepath"])
        np.load(row["trans_node_config_filepath"])

# データクラスを定義


In [8]:
@dataclass
class CatStatus:
    dfcat: pd.DataFrame
    prefix: str
    num_cat_dict: dict[str, int] = field(init=False)
    index_dict: dict[str, list[int]] = field(init=False)

    def __post_init__(self) -> None:
        self.num_cat_dict, self.index_dict = {}, {}
        dim_start = 0
        for i, row in self.dfcat.iterrows():
            self.num_cat_dict[f"{self.prefix}cat_feat{i + 1}"] = row["num_cats"]
            self.index_dict[f"{self.prefix}cat_feat{i + 1}"] = list(
                range(dim_start, dim_start + row["num_dims"])
            )
            dim_start += row["num_dims"]


cat_status = CatStatus(dfcat=dfcat, prefix="")
cat_config_status = CatStatus(dfcat=dfcat_config, prefix="config_")


@dataclass
class Const:
    num_node_flag_feat_dim: int
    num_node_cont_feat_dim: int
    num_node_cat_feat_dim: int
    num_node_config_cont_feat_dim: int

    # 演算子の種類
    num_operations: int = 120
    # 各configの次元数
    num_config_dims: int = 6


fileobj = np.load(dataset_dict["train"].iloc[0]["node_feat_filepath"])
trans_fileobj = np.load(dataset_dict["train"].iloc[0]["trans_node_feat_filepath"])
trans_config_fileobj = np.load(
    dataset_dict["train"].iloc[0]["trans_node_config_filepath"]
)

node_flag_feat, node_cont_feat = fileobj["node_flag_feat"], fileobj["node_cont_feat"]
node_enum_feat, node_dimension_number_feat = (
    fileobj["node_enum_feat"],
    fileobj["node_dimension_number_feat"],
)
trans_node_cont_feat = trans_fileobj["node_feat"]
trans_node_config_cont_feat = trans_config_fileobj["node_config_cont_feat"]
const = Const(
    num_node_flag_feat_dim=node_flag_feat.shape[1] + 1,  # config_idsの分+1
    num_node_cont_feat_dim=node_cont_feat.shape[1] + trans_node_cont_feat.shape[1],
    num_node_cat_feat_dim=node_enum_feat.shape[1] + node_dimension_number_feat.shape[1],
    num_node_config_cont_feat_dim=trans_node_config_cont_feat.shape[2],
)


@dataclass
class NodeFeatExtractor:
    dims: list[int] = field(default_factory=lambda: [64, 64])
    leakyrelu_negative_slope: float = 0.1
    dropout_p: float = 0.2


@dataclass
class GNNExtractor:
    dims: list[int] = field(default_factory=lambda: [64, 64])
    leakyrelu_negative_slope = 0.1
    dropout_p: float = 0.2


@dataclass
class CatEmbedding:
    num_cat: int
    embedding_dim: int


@dataclass
class Params:
    device: str
    cat_embeddings: dict[str, CatEmbedding]
    # num_random_samples: int = 30
    # num_sampling: int = 33  # 1グラフ毎に1000件（テストと同じ）
    # train_batch_size: int = 8
    # batch_size: int = 30
    num_random_samples: int = 15
    num_sampling: int = 10  # 1グラフ毎に1000件（テストと同じ）
    train_batch_size: int = 1
    batch_size: int = 30
    node_feat_extractor: NodeFeatExtractor = field(
        default_factory=lambda: NodeFeatExtractor()
    )
    node_config_feat_extractor: NodeFeatExtractor = field(
        default_factory=lambda: NodeFeatExtractor()
    )
    gnn_extractor: GNNExtractor = field(default_factory=lambda: GNNExtractor())
    subgraph_extractor: NodeFeatExtractor = field(
        default_factory=lambda: NodeFeatExtractor()
    )
    epoch: int = 50
    T_max: int = 50
    eta_min: float = 1e-5
    lr: float = 1e-2
    weight_decay: float = 0


cat_embeddings = {}
cat_embeddings.update(
    {"op": CatEmbedding(num_cat=const.num_operations, embedding_dim=16)}
)
cat_embeddings.update(
    {
        k: CatEmbedding(num_cat=v, embedding_dim=16)
        for k, v in cat_status.num_cat_dict.items()
    }
)
cat_embeddings.update(
    {
        k: CatEmbedding(num_cat=v, embedding_dim=16)
        for k, v in cat_config_status.num_cat_dict.items()
    }
)
params = Params(
    device="cuda" if torch.cuda.is_available() else "cpu",
    cat_embeddings=cat_embeddings,
)


@dataclass
class LayoutConfigs:
    """
    Attributes
    ----------
    node_cont_feat: np.ndarray
        ノード特徴量、(ノード数, 108)

    node_cat_feat: np.ndarray
        ノード特徴量、(ノード数, 3)

    node_opcode: np.ndarray
        ノード演算子、(ノード数,)
    edge_index: np.ndarray
        エッジ、(エッジ数, 2)

    node_config_feat: np.ndarray
        設定毎のノード特徴量、(設定数, 設定可能なノード数, 3)

    node_config_ids: np.ndarray
        設定可能なノードのIndex、(設定可能なノード数,)
    config_runtime: np.ndarray
        実行時間、(設定数,)
    node_splits: np.ndarray
        同じパーティションでの計算を意味する。今回は使用しない。(パーティション数, 2)
    """

    node_flag_feat: np.ndarray
    node_cont_feat: np.ndarray
    node_cat_feat: np.ndarray
    node_opcode: np.ndarray
    edge_index: np.ndarray
    node_config_feat: np.ndarray
    node_config_cont_feat: np.ndarray
    node_config_ids: np.ndarray
    config_runtime: np.ndarray
    node_splits: np.ndarray

    cat_status: CatStatus
    cat_config_status: CatStatus
    target: np.ndarray = field(init=False)

    def __post_init__(self) -> None:
        # 設定が存在するノードのフラグ
        node_active_feat = np.zeros((self.num_nodes, 1))
        node_active_feat[self.node_config_ids, :] = 1
        self.node_flag_feat = np.concatenate(
            [self.node_flag_feat, node_active_feat], axis=1
        )
        self.node_cont_feat = self.apply_normalization(x=self.node_cont_feat)
        self.node_config_feat = self.node_config_feat + 1  # カテゴリは0~7にする
        self.node_splits = np.array(
            [
                [self.node_splits[0][i], self.node_splits[0][i + 1] - 1]
                for i in range(self.node_splits.shape[1] - 1)
            ]
        )
        self.target = self.apply_target_normalization(x=self.config_runtime)

    @property
    def num_nodes(self) -> int:
        """ノード数"""
        return self.node_cont_feat.shape[0]

    def get_random_config_idx(self, num: int) -> list[int]:
        """ランダムな設定をサンプルする"""
        num_ = min(self.config_runtime.shape[0], num)
        return random.sample(list(range(self.config_runtime.shape[0])), num_)

    def get_filled_node_config_feat(
        self, index_list: list[int]
    ) -> tuple[np.ndarray, np.ndarray]:
        """指定された設定の設定毎のノード特徴量を取得する。設定がない場合は補完する。
        Parameters
        ----------
        index_list: list[int]
            設定のIndex

        Returns
        -------
        np.ndarray [(len(index_list),ノード数, 18), (len(index_list),ノード数, 連続次元数)]
        """
        # (サンプル数, ノード数) x 3
        node_config_feat = np.full(
            (len(index_list), self.num_nodes, Const.num_config_dims * 3),
            Const.num_config_dims + 1,
        )
        node_config_feat[:, self.node_config_ids] = self.node_config_feat[
            index_list, :, :
        ]

        node_config_cont_feat = np.zeros(
            (len(index_list), self.num_nodes, self.node_config_cont_feat.shape[2])
        )
        node_config_cont_feat[:, self.node_config_ids] = self.node_config_cont_feat[
            index_list, :, :
        ]
        return node_config_feat, node_config_cont_feat

    def get_target(self, index_list: list[int]) -> np.ndarray:
        """指定された設定の目的変数を取得する

        Parameters
        ----------
        index_list: list[int]
            設定のIndex

        Returns
        -------
        np.ndarray
        """
        return self.apply_target_ranking(x=self.config_runtime[index_list])

    def apply_normalization(self, x: np.ndarray) -> np.ndarray:
        """特徴量の正規化

        Parameters
        ----------
        x: np.ndarray
            2次元行列

        Returns
        -------
        x: np.ndarray
            行方向に正規化された行列
        """
        return np.log1p(x / 128)

    def apply_target_normalization(self, x: np.ndarray) -> np.ndarray:
        """目的変数の正規化

        Parameters
        ----------
        x: np.ndarray
            ベクトル

        Returns
        -------
        x: np.ndarray
            正規化されたベクトル
        """
        return np.log(x / x.min())

    def apply_target_ranking(self, x: np.ndarray) -> np.ndarray:
        """降順でランキング"""
        return np.argsort(np.argsort(-x))

## データセットを定義


In [9]:
class LayoutDataset(Dataset):
    """
    Attributes
    ----------
    rows: list[dict[str, np.ndarray]]
        設定をリストでもつ
    """

    def __init__(
        self,
        dataset: pd.DataFrame,
        params: Params,
        cat_status: CatStatus,
        cat_config_status: CatStatus,
    ) -> None:
        self.rows = dataset.to_dict("records")
        self.params = params
        self.cat_status = cat_status
        self.cat_config_status = cat_config_status
        self.cache_idx = None
        self.cache_filepath = None

    @property
    def device(self) -> str:
        return self.params.device

    def __len__(self) -> int:
        return len(self.rows)

    def create_layout_config(self, idx: int) -> LayoutConfigs:
        if self.cache_idx != idx:
            self.cache_idx = idx
            fileobj = np.load(self.rows[self.cache_idx]["filepath"])
            node_feat_fileobj = np.load(self.rows[self.cache_idx]["node_feat_filepath"])
            trans_feat_fileobj = np.load(
                self.rows[self.cache_idx]["trans_node_feat_filepath"]
            )
            trans_config_feat_fileobj = np.load(
                self.rows[self.cache_idx]["trans_node_config_filepath"]
            )

            node_cont_feat = np.concatenate(
                [
                    node_feat_fileobj["node_cont_feat"],
                    trans_feat_fileobj["node_feat"],
                ],
                axis=1,
            )

            node_cat_feat = np.concatenate(
                [
                    node_feat_fileobj["node_enum_feat"],
                    node_feat_fileobj["node_dimension_number_feat"],
                ],
                axis=1,
            )

            self.cache_layout_config = LayoutConfigs(
                node_opcode=fileobj["node_opcode"],
                edge_index=fileobj["edge_index"],
                node_config_ids=fileobj["node_config_ids"],
                config_runtime=fileobj["config_runtime"],
                node_splits=fileobj["node_splits"],
                node_flag_feat=node_feat_fileobj["node_flag_feat"],
                node_cont_feat=node_cont_feat,
                node_cat_feat=node_cat_feat,
                node_config_feat=fileobj["node_config_feat"],
                node_config_cont_feat=trans_config_feat_fileobj[
                    "node_config_cont_feat"
                ],
                cat_status=self.cat_status,
                cat_config_status=self.cat_config_status,
            )
        return self.cache_layout_config

    def __getitem__(
        self, idx: int
    ) -> tuple[
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
    ]:
        """ランダムな設定を取得する"""
        layout_configs = self.create_layout_config(idx=idx)
        config_index_list = layout_configs.get_random_config_idx(
            num=self.params.num_random_samples
        )
        return self._get_tensors(
            layout_configs=layout_configs, index_list=config_index_list
        )

    def getitem_as_batch(
        self, idx: int
    ) -> list[
        tuple[
            torch.Tensor,
            torch.Tensor,
            torch.Tensor,
            torch.Tensor,
            torch.Tensor,
            torch.Tensor,
        ]
    ]:
        """設定をバッチで取得する"""
        layout_configs = self.create_layout_config(idx=idx)

        index_list = list(range(layout_configs.config_runtime.shape[0]))
        for i_chunk in range(0, len(index_list), self.params.batch_size):
            chunk_index_list = index_list[i_chunk : i_chunk + self.params.batch_size]
            yield self._get_tensors(
                layout_configs=layout_configs, index_list=chunk_index_list
            )

    def _get_tensors(
        self, layout_configs: LayoutConfigs, index_list: list[int]
    ) -> tuple[
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
    ]:
        """渡された設定のIndexのテンソルを取得する

        Parameters
        ----------
        layout_configs: LayoutConfigs
            Layoutのデータクラス
        index_list: list[int]
            設定のインデックス

        Returns
        -------
        torch.Tensor
            ノード特徴量(フラグ)
        torch.Tensor
            ノード特徴量(連続)
        dict[str, torch.Tensor]
            ノード特徴量(カテゴリ)
        torch.Tensor
            設定毎のノード特徴量
        torch.Tensor
            設定毎のノード特徴量(連続)
        torch.Tensor
            ノード演算子
        torch.Tensor
            エッジ
        torch.Tensor
            目的変数
        """
        # ノード特徴量(フラグ)
        node_flag_feat = torch.tensor(
            layout_configs.node_flag_feat,
            dtype=torch.float32,
        ).to(self.device)
        # ノード特徴量(連続)
        node_cont_feat = torch.tensor(
            layout_configs.node_cont_feat,
            dtype=torch.float32,
        ).to(self.device)
        # ノード特徴量(カテゴリ)
        node_cat_feat = torch.tensor(
            layout_configs.node_cat_feat,
            dtype=torch.int64,
        ).to(self.device)
        # 設定毎のノード特徴量(カテゴリ)
        (
            node_config_feat,
            node_config_cont_feat,
        ) = layout_configs.get_filled_node_config_feat(index_list=index_list)
        node_config_feat = torch.tensor(node_config_feat, dtype=torch.int64).to(
            self.device
        )
        node_config_cont_feat = torch.tensor(
            node_config_cont_feat, dtype=torch.float32
        ).to(self.device)
        # ノード演算子
        node_opcode = torch.tensor(layout_configs.node_opcode, dtype=torch.int64).to(
            self.device
        )
        # エッジ
        edge_index = torch.tensor(
            np.swapaxes(layout_configs.edge_index, 0, 1), dtype=torch.int64
        ).to(self.device)
        # サブグラフ
        node_splits = torch.tensor(layout_configs.node_splits, dtype=torch.int64).to(
            self.device
        )
        # ターゲット
        target = torch.tensor(
            layout_configs.get_target(index_list=index_list),
            dtype=torch.float32,
        ).to(self.device)

        return (
            node_opcode,
            node_flag_feat,
            node_cont_feat,
            node_cat_feat,
            node_config_feat,
            node_config_cont_feat,
            edge_index,
            node_splits,
            target,
        )

    def get_ith_file_info(self, i: int) -> dict[str, str]:
        row = self.rows[i]
        return {
            "arch": row["arch"],
            "perm": row["perm"],
            "filename": row["filename"],
        }

    def get_ith_runtime(self, i: int) -> np.ndarray:
        layout_configs = self.create_layout_config(idx=i)
        return layout_configs.config_runtime

## モデルを定義


In [10]:
from torch_geometric.nn import MessagePassing


class EdgeConv(MessagePassing):
    """
    ノード特徴 + 隣接ノード特徴 + 隣接ノード特徴の一致
    参考： https://pytorch-geometric.readthedocs.io/en/latest/tutorial/create_gnn.html#implementing-the-edge-convolution
    補足: 集約関数はデフォルトでdim(axis) = -2。つまりノード方向で集約するので気にしなくてOK
    https://github.com/pyg-team/pytorch_geometric/blob/1e12d41c28b1fb9793f17646b018071b508864d7/torch_geometric/nn/aggr/basic.py#L38
    """

    def __init__(self, x_input_dim: int, x_output_dim: int, dropout_p: float):
        # "Add" aggregation
        super().__init__(aggr="max")
        self.mlp = nn.Sequential(
            # nn.LayerNorm(x_input_dim * 2),
            nn.Linear(x_input_dim * 2, x_output_dim),
            # nn.Dropout(dropout_p),
            nn.ReLU(),
            # nn.LayerNorm(x_output_dim),
            nn.Linear(x_output_dim, x_output_dim),
            # nn.Dropout(dropout_p),
        )

    def forward(self, x, edge_index):
        # x has shape [設定数, N, in_channels]
        # edge_index has shape [2, E]
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        """propagate()で渡された引数xから自動でx_i, x_jノードを取り出して随時処理を実装する関数"""
        # x_i has shape [設定数, エッジ数, in_channels]
        # x_j has shape [設定数, エッジ数, in_channels]
        x_cat = torch.cat(
            [x_i, x_i - x_j], dim=2
        )  # tmp has shape [設定数, エッジ数, 2 * in_channels]
        return self.mlp(x_cat)


class SimpleLayoutModel(torch.nn.Module):
    """

    Attributes
    ----------
    params: Params
        実験設定のデータクラス
    node_embeddings: torch.Tensor
        カテゴリ変数の埋め込み表現(ノード毎)
    node_config_embeddings: torch.Tensor
        カテゴリ変数の埋め込み表現(設定xノード毎)
    node_feat_extractor: torch.nn.Module
        ノードの特徴量を抽出するネットワーク
    gnn_extractor: torch.nn.Module
        グラフの特徴量を抽出するネットワーク
    gc: torch.nn.Module
        最終層の全結合層
    """

    def __init__(
        self,
        params: Params,
        const: Const,
        cat_status: CatStatus,
        cat_config_status: CatStatus,
    ) -> None:
        super().__init__()
        self.params = params
        self.cat_status = cat_status
        self.cat_config_status = cat_config_status

        # カテゴリ変数の埋め込み表現
        self.embeddings = nn.ModuleDict(
            {
                k: torch.nn.Embedding(v.num_cat, v.embedding_dim)
                for k, v in self.params.cat_embeddings.items()
            }
        )

        # node_featのfeature_extractorを定義
        num_node_feat_extractor_input_dim = (
            const.num_node_flag_feat_dim
            + const.num_node_cont_feat_dim
            + self.num_node_feat_embedding_dims
        )

        node_feat_extractor_layer = []
        node_feat_extractor_dims = [
            num_node_feat_extractor_input_dim
        ] + self.params.node_feat_extractor.dims
        for i in range(len(node_feat_extractor_dims) - 1):
            node_feat_extractor_layer += [
                # nn.LayerNorm(node_feat_extractor_dims[i]),
                nn.Linear(
                    in_features=node_feat_extractor_dims[i],
                    out_features=node_feat_extractor_dims[i + 1],
                ),
                # nn.Dropout(params.node_feat_extractor.dropout_p),
                nn.LeakyReLU(params.node_feat_extractor.leakyrelu_negative_slope),
            ]
            self.node_feat_extractor = nn.Sequential(*node_feat_extractor_layer)

        # node_config_featのfeature_extractorを定義
        num_node_config_feat_extractor_input_dim = (
            self.num_node_config_feat_embedding_dims
            + const.num_node_config_cont_feat_dim
        )

        node_config_feat_extractor_layer = []
        node_config_feat_extractor_dims = [
            num_node_config_feat_extractor_input_dim
        ] + self.params.node_config_feat_extractor.dims
        for i in range(len(node_feat_extractor_dims) - 1):
            node_config_feat_extractor_layer += [
                # nn.LayerNorm(node_config_feat_extractor_dims[i]),
                nn.Linear(
                    in_features=node_config_feat_extractor_dims[i],
                    out_features=node_config_feat_extractor_dims[i + 1],
                ),
                # nn.Dropout(params.node_config_feat_extractor.dropout_p),
                nn.LeakyReLU(
                    params.node_config_feat_extractor.leakyrelu_negative_slope
                ),
            ]
        self.node_config_feat_extractor = nn.Sequential(
            *node_config_feat_extractor_layer
        )

        # ノード間のfeature_extractorの定義
        num_gnn_extractor_input_dim = (
            node_feat_extractor_dims[-1] + node_config_feat_extractor_dims[-1]
        )

        gnn_extractor_layer = []
        gnn_extractor_dims = [
            num_gnn_extractor_input_dim
        ] + self.params.gnn_extractor.dims
        for i in range(len(gnn_extractor_dims) - 1):
            gnn_extractor_layer += [
                (
                    EdgeConv(
                        x_input_dim=gnn_extractor_dims[i],
                        x_output_dim=gnn_extractor_dims[i + 1],
                        dropout_p=params.gnn_extractor.dropout_p,
                    ),
                    "x, edge_index -> x",
                ),
                nn.LeakyReLU(params.gnn_extractor.leakyrelu_negative_slope),
            ]
        self.gnn_extractor = Sequential("x, edge_index", gnn_extractor_layer)

        # サブグラフのfeature_extractorの定義
        # num_subgraph_extractor_input_dim = (
        #     self.params.gnn_extractor.dims[-1] + num_gnn_extractor_input_dim
        # )

        # subgraph_extractor_layer = []
        # subgraph_extractor_dims = [
        #     num_subgraph_extractor_input_dim
        # ] + self.params.node_feat_extractor.dims
        # for i in range(len(subgraph_extractor_dims) - 1):
        #     subgraph_extractor_layer += [
        #         # nn.LayerNorm(subgraph_extractor_dims[i]),
        #         nn.Linear(
        #             in_features=subgraph_extractor_dims[i],
        #             out_features=subgraph_extractor_dims[i + 1],
        #         ),
        #         # nn.Dropout(params.subgraph_extractor.dropout_p),
        #         nn.LeakyReLU(params.subgraph_extractor.leakyrelu_negative_slope),
        #     ]
        # self.subgraph_extractor = nn.Sequential(*subgraph_extractor_layer)

        fc_layer = [
            # nn.LayerNorm(subgraph_extractor_dims[-1]),
            # nn.Linear(in_features=subgraph_extractor_dims[-1], out_features=1),
            nn.Linear(
                in_features=self.params.gnn_extractor.dims[-1]
                + num_gnn_extractor_input_dim,
                out_features=1,
            ),
        ]
        self.fc = nn.Sequential(*fc_layer)
        self.to(self.params.device)

    @property
    def num_node_feat_embedding_dims(self) -> int:
        num_embedding_dims = 0
        num_embedding_dims += 1 * self.params.cat_embeddings["op"].embedding_dim
        for cat_name, cat_index in self.cat_status.index_dict.items():
            num_embedding_dims += (
                len(cat_index) * self.params.cat_embeddings[cat_name].embedding_dim
            )
        return num_embedding_dims

    @property
    def num_node_config_feat_embedding_dims(self) -> int:
        num_embedding_dims = 0
        for cat_name, cat_index in self.cat_config_status.index_dict.items():
            num_embedding_dims += (
                len(cat_index) * self.params.cat_embeddings[cat_name].embedding_dim
            )
        return num_embedding_dims

    def forward(
        self,
        node_opcode: torch.Tensor,
        node_flag_feat: torch.Tensor,
        node_cont_feat: torch.Tensor,
        node_cat_feat: torch.Tensor,
        node_config_feat: torch.Tensor,
        node_config_cont_feat: torch.Tensor,
        edge_index: torch.Tensor,
        node_splits: torch.Tensor,
    ) -> torch.Tensor:
        """
        Parameters
        ------
        node_flag_feat:
            ノードの特徴量(node数, フラグ次元数)
        node_cont_feat:
            ノードの特徴量(node数, 連続次元数)
        node_cat_feat:
            ノードの特徴量(node数, カテゴリ次元数*埋め込み次元数)
        node_config_feat:
            設定毎のノードの特徴量(設定数, node数, 特徴次元数)
        node_config_cont_feat:
            設定毎のノードの特徴量(設定数, node数, 連続次元数)
        edge_index:
            エッジ(2, エッジ数)
        node_splits:
            サブグラフのインデックス（サブグラフ数, 2)

        Returns:
        torch.tensor: (設定数)
        """
        # (ノード数,特徴数)のテンソルを作成
        node_feat = self._join_node_feature(
            node_opcode=node_opcode,
            node_flag_feat=node_flag_feat,
            node_cont_feat=node_cont_feat,
            node_cat_feat=node_cat_feat,
        )

        # (設定数,ノード数,特徴数)のテンソルを作成
        node_config_feat = self._join_node_config_feature(
            node_config_feat=node_config_feat,
            node_config_cont_feat=node_config_cont_feat,
        )

        # node_featの抽出器を通す
        extracted_node_feat = self.node_feat_extractor(node_feat)

        # node_config_featの抽出器を通す
        extracted_node_config_feat = self.node_config_feat_extractor(node_config_feat)

        # 設定毎のノード特徴に結合する
        extracted_feat = self._join_entire_node_config_feat(
            node_feat=extracted_node_feat,
            node_config_feat=extracted_node_config_feat,
        )

        # GNN抽出器を通す
        conved_extracted_feat = self.gnn_extractor(
            x=extracted_feat,
            edge_index=edge_index,
        )

        # 残差を足すイメージ
        concat_feat = torch.concat([extracted_feat, conved_extracted_feat], 2)

        # subgraph_global_pool_feat_list = []
        # for subgraph_start_node_idx, subgraph_end_node_idx in node_splits:
        #     subgraph_concat_feat = concat_feat[
        #         :, subgraph_start_node_idx : subgraph_end_node_idx + 1, :
        #     ]
        #     # ノードの特徴量を足し合わせる(Global mean Pooling)
        #     subgraph_global_pool_feat = torch.mean(concat_feat, dim=1)
        #     subgraph_global_pool_feat_list.append(
        #         torch.reshape(
        #             subgraph_global_pool_feat,
        #             (
        #                 subgraph_global_pool_feat.shape[0],
        #                 1,
        #                 subgraph_global_pool_feat.shape[1],
        #             ),
        #         )
        #     )
        # # （設定数,サブグラフ数,特徴数)
        # subgraph_global_pool_feat = torch.concat(subgraph_global_pool_feat_list, 1)
        # subgraph_extracted_feat = self.subgraph_extractor(subgraph_global_pool_feat)

        # ノードの特徴量を足し合わせる(Global mean Pooling)
        # global_pool_feat = torch.mean(subgraph_extracted_feat, dim=1)
        global_pool_feat = torch.mean(concat_feat, dim=1)

        return torch.squeeze(self.fc(global_pool_feat))

    def _join_node_feature(
        self,
        node_opcode: torch.Tensor,
        node_flag_feat: torch.Tensor,
        node_cont_feat: torch.Tensor,
        node_cat_feat: torch.Tensor,
    ) -> torch.Tensor:
        """node_featのテンソルを作成"""
        # ノードの埋め込み表現
        node_embeddings_list = []
        node_embeddings_list.append(self.embeddings["op"](node_opcode))
        for cat_name, cat_index in self.cat_status.index_dict.items():
            node_embeddings = self.embeddings[cat_name](node_cat_feat[:, cat_index])
            node_embeddings = torch.reshape(
                node_embeddings,
                (-1, node_embeddings.shape[-2] * node_embeddings.shape[-1]),
            )
            node_embeddings_list.append(node_embeddings)

        # ノード毎で埋め込み、結合(ノード数, 特徴数)
        node_embedding_feat = torch.concat(node_embeddings_list, 1)
        node_feat = torch.concat(
            [node_flag_feat, node_cont_feat, node_embedding_feat], 1
        )
        return node_feat

    def _join_node_config_feature(
        self, node_config_feat: torch.Tensor, node_config_cont_feat: torch.Tensor
    ) -> torch.Tensor:
        """node_config_featのテンソルを作成"""
        # 設定xノード毎で埋め込み(設定数, ノード数, 特徴数)
        node_config_embeddings_list = []
        for cat_name, cat_index in self.cat_config_status.index_dict.items():
            node_embeddings = self.embeddings[cat_name](
                node_config_feat[:, :, cat_index]
            )
            node_embeddings = torch.reshape(
                node_embeddings,
                (
                    node_embeddings.shape[0],
                    -1,
                    node_embeddings.shape[-2] * node_embeddings.shape[-1],
                ),
            )
            node_config_embeddings_list.append(node_embeddings)
        node_config_feat = torch.concat(
            node_config_embeddings_list + [node_config_cont_feat], 2
        )
        return node_config_feat

    def _join_entire_node_config_feat(
        self, node_feat: torch.Tensor, node_config_feat: torch.Tensor
    ) -> torch.Tensor:
        # ノード毎の特徴量を設定数だけ縦に並べる
        node_tiled_feat = torch.tile(
            torch.reshape(node_feat, (1, node_feat.shape[0], node_feat.shape[1])),
            (node_config_feat.shape[0], 1, 1),
        )
        return torch.concat([node_tiled_feat, node_config_feat], 2)

## 学習


In [11]:
class ListMLE(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, logits: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """

        Parameters
        ----------
        logits: torch.Tensor
            予測（要素数, ）
        labels: torch.Tensor
            目的変数（要素数, ）

        Returns
        -------
        torch.Tensor
        """
        # 正解をソート
        labels_sorted, labels_sorted_indice = labels.sort(descending=True, dim=1)
        # 予測を正解順でソート
        logits_sorted_by_true = torch.gather(logits, dim=1, index=labels_sorted_indice)
        # 予測値の最大値で予測値を引く（expの爆発予防）
        logits_max, _ = logits_sorted_by_true.max(dim=1, keepdim=True)
        logits_sorted_by_true = logits_sorted_by_true - logits_max
        # ランキングが低いものから累積する(その後正解順に戻す)
        cumsums = torch.cumsum(logits_sorted_by_true.exp().flip(dims=[1]), dim=1).flip(
            dims=[1]
        )
        # 誤差
        negative_log_likelihood = torch.sum(
            torch.log(cumsums) - logits_sorted_by_true, dim=1
        )
        return torch.mean(negative_log_likelihood)


def rankNet(y_pred, y_true):
    """
    RankNet loss introduced in "Learning to Rank using Gradient Descent".
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :return: loss value, a torch.Tensor
    """
    y_pred = y_pred.clone()
    y_true = y_true.clone()

    # here we generate every pair of indices from the range of document length in the batch
    document_pairs_candidates = list(
        itertools.product(range(y_true.shape[1]), repeat=2)
    )

    pairs_true = y_true[:, document_pairs_candidates]
    selected_pred = y_pred[:, document_pairs_candidates]

    # here we calculate the relative true relevance of every candidate pair
    true_diffs = pairs_true[:, :, 0] - pairs_true[:, :, 1]
    pred_diffs = selected_pred[:, :, 0] - selected_pred[:, :, 1]

    # here we filter just the pairs that are 'positive' and did not involve a padded instance
    # we can do that since in the candidate pairs we had symetric pairs so we can stick with
    # positive ones for a simpler loss function formulation
    the_mask = (true_diffs > 0) & (~torch.isinf(true_diffs))

    pred_diffs = pred_diffs[the_mask]

    weight = None
    # here we 'binarize' true relevancy diffs since for a pairwise loss we just need to know
    # whether one document is better than the other and not about the actual difference in
    # their relevancy levels
    true_diffs = (true_diffs > 0).type(torch.float32)
    true_diffs = true_diffs[the_mask]

    return nn.BCEWithLogitsLoss(weight=weight)(pred_diffs, true_diffs)


def to_cpu_numpy(
    params: Params, pred: torch.Tensor, truth: torch.Tensor
) -> tuple[np.ndarray, np.ndarray]:
    if params.device == "cuda":
        pred_ = pred.cpu().detach().numpy()
        truth_ = truth.cpu().detach().numpy()
        torch.cuda.empty_cache()
    else:
        pred_ = pred.detach().numpy()
        truth_ = truth.detach().numpy()
    return pred_, truth_

In [12]:
from scipy.stats import kendalltau


def evaluate_score(dataset: LayoutDataset, model: torch.nn.Module) -> pd.DataFrame:
    """データセット全件に対してコンペの評価指標を算出する
    https://www.kaggle.com/competitions/predict-ai-model-runtime/overview
    """
    model.eval()
    # criterion = ListMLE()

    records = []
    # 各グラフ毎にスコアを算出
    for i in range(len(dataset)):
        # グラフ毎に複数サンプル
        preds, truths = [], []
        current_loss = 0
        for _ in range(params.num_sampling):
            (
                node_opcode,
                node_flag_feat,
                node_cont_feat,
                node_cat_feat,
                node_config_feat,
                node_config_cont_feat,
                edge_index,
                node_splits,
                target,
            ) = dataset[i]
            pred = model(
                node_opcode=node_opcode,
                node_flag_feat=node_flag_feat,
                node_cont_feat=node_cont_feat,
                node_cat_feat=node_cat_feat,
                node_config_feat=node_config_feat,
                node_config_cont_feat=node_config_cont_feat,
                edge_index=edge_index,
                node_splits=node_splits,
            )
            # loss = criterion(
            #     torch.reshape(pred, (1, pred.shape[0])),
            #     torch.reshape(target, (1, target.shape[0])),
            # )
            loss = rankNet(
                torch.reshape(pred, (1, pred.shape[0])),
                torch.reshape(target, (1, target.shape[0])),
            )
            current_loss += loss.item()
            pred, truth = to_cpu_numpy(params, pred, target)
            preds.append(pred)
            truths.append(truth)
        preds, truths = np.hstack(preds), np.hstack(truths)
        current_loss /= params.num_sampling

        score = kendalltau(truth, pred).correlation
        record = dataset.get_ith_file_info(i)
        record.update(
            {
                "current_loss": current_loss,
                "score": score,
            }
        )
        records.append(record)
    return pd.DataFrame(records)

### 学習


In [13]:
def get_random_indexes_as_batch(num_data: int, batch_size: int) -> list[list[int]]:
    random_indexes = random.sample(list(range(num_data)), num_data)
    batches = []
    batch_random_indexes = []
    for i, index in enumerate(random_indexes):
        batch_random_indexes.append(index)
        if (i + 1) % batch_size == 0:
            batches.append(batch_random_indexes)
            batch_random_indexes = []
    if len(batch_random_indexes) > 0:
        batches.append(
            batch_random_indexes
            + random_indexes[: (batch_size - len(batch_random_indexes))]
        )
    return batches


def train_model(
    dftrain: pd.DataFrame,
    dfvalid: pd.DataFrame,
    params: Params,
    const: Const,
    cat_status: CatStatus,
    cat_config_status: CatStatus,
    savedir: Path,
    checkpoint_dir: Path = None,
) -> None:
    train_layout_dataset = LayoutDataset(
        dataset=dftrain,
        params=params,
        cat_status=cat_status,
        cat_config_status=cat_config_status,
    )
    valid_layout_dataset = LayoutDataset(
        dataset=dfvalid,
        params=params,
        cat_status=cat_status,
        cat_config_status=cat_config_status,
    )

    model = SimpleLayoutModel(
        params=params,
        const=const,
        cat_status=cat_status,
        cat_config_status=cat_config_status,
    )
    if checkpoint_dir is not None:
        print("学習済みモデルを読み込みます")
        model.load_state_dict(torch.load(checkpoint_dir / f"final_model.pt"))

    optimizer = torch.optim.Adam(
        model.parameters(), lr=params.lr, weight_decay=params.weight_decay
    )
    scheduler = CosineAnnealingLR(
        optimizer=optimizer, T_max=params.T_max, eta_min=params.eta_min
    )
    # criterion = ListMLE()

    best_score = -np.inf
    records = []
    for epoch in range(params.epoch):
        model.train()

        num_data = len(train_layout_dataset)
        pbar = tqdm(range(num_data))
        batch_indexes_list = get_random_indexes_as_batch(
            num_data=num_data, batch_size=params.train_batch_size
        )
        num_batch = len(batch_indexes_list)
        num_total_data = sum(
            [len(batch_indexes) for batch_indexes in batch_indexes_list]
        )
        i_total_data = 1

        running_losses = []
        for i_batch, batch_indexes in enumerate(batch_indexes_list):
            # 各グラフのループ
            for i_data in batch_indexes:
                # グラフの中で複数サンプルする
                preds, truths = [], []
                current_loss = 0
                for _ in range(params.num_sampling):
                    (
                        node_opcode,
                        node_flag_feat,
                        node_cont_feat,
                        node_cat_feat,
                        node_config_feat,
                        node_config_cont_feat,
                        edge_index,
                        node_splits,
                        target,
                    ) = train_layout_dataset[i_data]
                    out = model(
                        node_opcode=node_opcode,
                        node_flag_feat=node_flag_feat,
                        node_cont_feat=node_cont_feat,
                        node_cat_feat=node_cat_feat,
                        node_config_feat=node_config_feat,
                        node_config_cont_feat=node_config_cont_feat,
                        edge_index=edge_index,
                        node_splits=node_splits,
                    )

                    # loss = criterion(
                    #     torch.reshape(out, (1, out.shape[0])),
                    #     torch.reshape(target, (1, target.shape[0])),
                    # )
                    loss = rankNet(
                        torch.reshape(out, (1, out.shape[0])),
                        torch.reshape(target, (1, target.shape[0])),
                    )
                    current_loss += loss.item()
                    # loss /= (params.num_sampling * params.train_batch_size)
                    loss.backward()

                    pred, truth = to_cpu_numpy(params, out, target)
                    preds.append(pred)
                    truths.append(truth)

                optimizer.step()
                scheduler.step(epoch + i_total_data / num_total_data)
                optimizer.zero_grad()
                i_total_data += 1

                preds, truths = np.hstack(preds), np.hstack(truths)
                score = kendalltau(truth, pred).correlation
                current_loss /= params.num_sampling
                running_losses.append(current_loss)
                running_loss = np.mean(running_losses)

                record = {"epoch": epoch, "batch": i_batch}
                record.update(train_layout_dataset.get_ith_file_info(i_data))
                record.update(
                    {
                        "train/running_loss": running_loss,
                        "train/current_loss": current_loss,
                        "train/score": score,
                        "lr": scheduler.get_last_lr()[0],
                    }
                )
                records.append(record)
                wandb.log(record)

                pbar.set_description(
                    f"running loss: {running_loss:.5f}, current loss: {current_loss:.5f} score: {score:.3f}"
                )
                pbar.update(1)

            # optimizer.step()
            # scheduler.step(epoch + i_batch / num_batch)
            # optimizer.zero_grad()

        model.eval()
        torch.cuda.empty_cache()

        dfscore = evaluate_score(dataset=valid_layout_dataset, model=model)
        avg_loss = dfscore["current_loss"].mean()
        avg_score = dfscore["score"].mean()
        for _, row_score in dfscore.iterrows():
            record = {
                "epoch": epoch,
                "batch": -1,
                "arch": row_score["arch"],
                "perm": row_score["perm"],
                "filename": row_score["filename"],
                "valid/running_loss": avg_loss,
                "valid/current_loss": row_score["current_loss"],
                "valid/score": row_score["score"],
                "lr": scheduler.get_last_lr()[0],
            }
            records.append(record)
            wandb.log(record)

        print(f"[valid] current loss: {avg_loss:.5f} score: {avg_score:.3f}")

        if best_score < avg_score:
            best_score = avg_score
            torch.save(model.state_dict(), savedir / "best_model.pt")
        torch.save(model.state_dict(), savedir / f"epoch{epoch + 1}_model.pt")

    dflog = pd.DataFrame(records)
    dflog.to_csv(savedir / "log.csv", index=False)

    torch.save(model.state_dict(), savedir / "final_model.pt")

    del (
        train_layout_dataset,
        valid_layout_dataset,
        model,
        optimizer,
        dfscore,
        dflog,
        records,
    )
    gc.collect()
    torch.cuda.empty_cache()

In [14]:
arch, perm = "nlp", "default"
exptname = str(Path().resolve()).split("/")[-1]

wandb.init(
    # set the wandb project where this run will be logged
    project="predict-ai-model-runtime-for-sun-scan-clan",
    # track hyperparameters and run metadata
    config={
        "params": asdict(params),
        "const": asdict(const),
        "arch": arch,
        "perm": perm,
        "validation": "hold-out",
    },
    name=exptname,
    tags=[arch, perm],
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: zuuuubo-tetsu (sun-scan-clan). Use `wandb login --relogin` to force relogin


In [15]:
dftrain = dataset_dict["train"].query(f"(arch == @arch) & (perm == @perm)")
dfvalid = dataset_dict["valid"].query(f"(arch == @arch) & (perm == @perm)")
params_opt = deepcopy(params)

savedir = workdir / f"{arch}-{perm}"
savedir.mkdir(exist_ok=True, parents=True)
train_model(
    dftrain=dftrain,
    dfvalid=dfvalid,
    params=params_opt,
    const=const,
    cat_status=cat_status,
    cat_config_status=cat_config_status,
    savedir=savedir,
    checkpoint_dir=None,
)
wandb.alert(title=exptname, text=f"Train End")

  0%|          | 0/198 [00:00<?, ?it/s]

[valid] current loss: 0.80348 score: 0.085


  0%|          | 0/198 [00:00<?, ?it/s]

In [24]:
train_layout_dataset = LayoutDataset(
    dataset=dftrain,
    params=params,
    cat_status=cat_status,
    cat_config_status=cat_config_status,
)

model = SimpleLayoutModel(
    params=params,
    const=const,
    cat_status=cat_status,
    cat_config_status=cat_config_status,
)
# model.load_state_dict(torch.load(workdir / "nlp-default" / f"_model.pt"))

In [25]:
(
    node_opcode,
    node_flag_feat,
    node_cont_feat,
    node_cat_feat,
    node_config_feat,
    node_config_cont_feat,
    edge_index,
    node_splits,
    target,
) = train_layout_dataset[1]
out = model(
    node_opcode=node_opcode,
    node_flag_feat=node_flag_feat,
    node_cont_feat=node_cont_feat,
    node_cat_feat=node_cat_feat,
    node_config_feat=node_config_feat,
    node_config_cont_feat=node_config_cont_feat,
    edge_index=edge_index,
    node_splits=node_splits,
)

In [26]:
target

tensor([ 6., 11.,  9., 14.,  4.,  0.,  5., 13.,  3., 10.,  1.,  8.,  2., 12.,
         7.], device='cuda:0')

In [27]:
out

tensor([0.0224, 0.0327, 0.0379, 0.0143, 0.0296, 0.0378, 0.0366, 0.0237, 0.0260,
        0.0363, 0.0382, 0.0366, 0.0368, 0.0314, 0.0242], device='cuda:0',
       grad_fn=<SqueezeBackward0>)

In [ ]:
savedir = workdir / f"{arch}-{perm}"
dflog = pd.read_csv(savedir / "log.csv")

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
for i, ds in enumerate(["train", "valid"]):
    dflog_ = dflog.query("(phase == @ds)").groupby("epoch")
    axes[i][0].plot(dflog_["current_loss"].mean(), label="total")
    axes[i][1].plot(dflog_["score"].mean(), label="total")
    if i == 0:
        axes[i][0].legend()
fig.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/yamaguchi/kaggle/experiments/1028-nlp-default/out/nlp-default/log.csv'

## 推論


In [ ]:
savedir = workdir / f"{arch}-{perm}"

records = []

dftest = dataset_dict["test"].query(f"(arch == @arch) & (perm == @perm)")
params_opt = deepcopy(params)
if arch == "nlp":
    params_opt.num_random_samples = 30
    params_opt.batch_size = 30

test_layout_dataset = LayoutDataset(
    dataset=dftest,
    params=params_opt,
    cat_status=cat_status,
    cat_config_status=cat_config_status,
)
model = SimpleLayoutModel(
    params=params_opt,
    const=const,
    cat_status=cat_status,
    cat_config_status=cat_config_status,
)
model.load_state_dict(torch.load(savedir / f"final_model.pt"))
model.eval()

with tqdm(range(len(test_layout_dataset))) as pbar:
    for i in pbar:
        file_info = test_layout_dataset.get_ith_file_info(i)

        pred_list = []
        for (
            node_opcode,
            node_flag_feat,
            node_cont_feat,
            node_cat_feat,
            node_config_feat,
            node_config_cont_feat,
            edge_index,
            node_splits,
            target,
        ) in test_layout_dataset.getitem_as_batch(i):
            pred_batch = model(
                node_opcode=node_opcode,
                node_flag_feat=node_flag_feat,
                node_cont_feat=node_cont_feat,
                node_cat_feat=node_cat_feat,
                node_config_feat=node_config_feat,
                node_config_cont_feat=node_config_cont_feat,
                edge_index=edge_index,
                node_splits=node_splits,
            )
            if params_opt.device == "cuda":
                pred_batch = pred_batch.cpu().detach().numpy()
            else:
                pred_batch = pred_batch.detach().numpy()
            # pred_batchは高いものほどよい
            pred_batch = -pred_batch
            pred_list.append(pred_batch)

            del (
                node_opcode,
                node_flag_feat,
                node_cont_feat,
                node_cat_feat,
                node_config_feat,
                node_config_cont_feat,
                edge_index,
                node_splits,
                target,
            )
            gc.collect()
            torch.cuda.empty_cache()

        pred = np.hstack(pred_list)

        ID = f"layout:{file_info['arch']}:{file_info['perm']}:{file_info['filename']}"
        records.append({"ID": ID, "pred": ";".join(list(map(str, pred.argsort())))})

del test_layout_dataset, model
gc.collect()
torch.cuda.empty_cache()

dfpred = pd.DataFrame(records)
dfsub = pd.read_csv(inputdir / "sample_submission.csv")
dfsub = dfsub.merge(dfpred, on="ID", how="left")
dfsub["TopConfigs"] = np.where(
    dfsub["pred"].isnull(), dfsub["TopConfigs"], dfsub["pred"]
)
dfsub[["ID", "TopConfigs"]].to_csv(savedir / f"submission_final_model.csv", index=False)

In [ ]:
wandb.alert(title=exptname, text=f"Inference End")
wandb.finish()